In [1]:
import arcpy
import os
from arcpy import env
from arcpy.sa import *
from arcpy.sa import Expand
from arcpy.sa import CreateConstantRaster

# Ensure Spatial Analyst extension is enabled
arcpy.CheckOutExtension("Spatial")

def expand_float_rasters(input_folder, output_folder, buffer_cells=20):
    
    """
    Expands a float raster by adding a buffer around the original raster.
    
    Parameters:
        input_folder (str): Folder containing input float rasters.
        output_folder (str): Folder to save expanded rasters.
        buffer_cells (int): Number of cells to add as a border on each side.
    """
    
    arcpy.env.workspace = input_folder
    arcpy.env.overwriteOutput = True

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List all rasters in the folder
    rasters = arcpy.ListRasters()

    for raster in rasters:
        try:
            # Collect data on original raster 
            desc = arcpy.Describe(raster)
            cell_size = desc.meanCellWidth
            sr = desc.spatialReference
            xmin = desc.extent.XMin
            ymin = desc.extent.YMin
            xmax = desc.extent.XMax
            ymax = desc.extent.YMax

            # Calculate new raster params 
            expand_dist = buffer_cells * cell_size
            new_xmin = xmin - expand_dist
            new_ymin = ymin - expand_dist
            new_xmax = xmax + expand_dist
            new_ymax = ymax + expand_dist

            arcpy.env.extent = f"{new_xmin} {new_ymin} {new_xmax} {new_ymax}"

            # Create new raster
            bg_raster = CreateConstantRaster(0, "FLOAT", cell_size, f"{new_xmin} {new_ymin} {new_xmax} {new_ymax}")
            bg_raster_name = "bg_" + raster.replace(".TIF", ".img")
            
            bg_raster_path = os.path.join(output_folder, bg_raster_name)
            bg_raster.save(bg_raster_path)
            
            out_raster_name = "expanded_" + raster
            out_raster_path = os.path.join(output_folder, out_raster_name)
            
            bands = desc.bandCount
            
            # Mosaic adds the original raster onto the constant background
            arcpy.MosaicToNewRaster_management(
                input_rasters=[raster, bg_raster_path],
                output_location=output_folder,
                raster_dataset_name_with_extension=out_raster_name,
                coordinate_system_for_the_raster=sr,
                pixel_type="32_BIT_FLOAT",
                cellsize=cell_size,
                number_of_bands=bands,
                mosaic_method="FIRST",
                mosaic_colormap_mode="FIRST"
            )
            
            print(f"Expanded raster saved: {out_raster_path}")

            # Enviroment managment
            arcpy.Delete_management(bg_raster_path)
            arcpy.env.extent = ""
        
        except Exception as e:
            print(f"Error processing {raster}: {e}")


In [2]:
#input_folder = r"C:\Users\sasha\Downloads\B_split_middle"
#output_folder = r"C:\Users\sasha\Downloads\B_split_middle_expanded"

input_folder = r"C:\Users\sasha\Downloads\T_split"
output_folder = r"C:\Users\sasha\Downloads\T_split_expanded"

expand_float_rasters(input_folder, output_folder, buffer_cells=40)

Expanded raster saved: C:\Users\sasha\Downloads\T_split_expanded\expanded_T_0.TIF
Expanded raster saved: C:\Users\sasha\Downloads\T_split_expanded\expanded_T_1.TIF
Expanded raster saved: C:\Users\sasha\Downloads\T_split_expanded\expanded_T_10.TIF
Expanded raster saved: C:\Users\sasha\Downloads\T_split_expanded\expanded_T_11.TIF
Expanded raster saved: C:\Users\sasha\Downloads\T_split_expanded\expanded_T_12.TIF
Expanded raster saved: C:\Users\sasha\Downloads\T_split_expanded\expanded_T_13.TIF
Expanded raster saved: C:\Users\sasha\Downloads\T_split_expanded\expanded_T_14.TIF
Expanded raster saved: C:\Users\sasha\Downloads\T_split_expanded\expanded_T_15.TIF
Expanded raster saved: C:\Users\sasha\Downloads\T_split_expanded\expanded_T_16.TIF
Expanded raster saved: C:\Users\sasha\Downloads\T_split_expanded\expanded_T_17.TIF
Expanded raster saved: C:\Users\sasha\Downloads\T_split_expanded\expanded_T_18.TIF
Expanded raster saved: C:\Users\sasha\Downloads\T_split_expanded\expanded_T_19.TIF
Expand